In [2]:
BASE_PATH = "/tf/notebooks/CEIA/computer-vision-2/tp-final/models/yolov7"

import os
import json

%pip install -r {os.path.join(BASE_PATH, "yolov7/requirements.txt")}

import torch

Note: you may need to restart the kernel to use updated packages.


In [3]:
PRETRAINED_PATH = os.path.join(BASE_PATH, "yolov7-pretrained")
WEIGHTS_FILE = os.path.join(PRETRAINED_PATH, "yolov7_training.pt")

if os.access(WEIGHTS_FILE, os.F_OK) is False:
    try:
        original_umask = os.umask(0)
        os.mkdir(PRETRAINED_PATH, mode=0o775)
    except FileExistsError:
        pass
    finally:
        os.umask(original_umask)

    # download COCO starting checkpoint
    !wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt -P {PRETRAINED_PATH}

In [5]:
RUNS_PATH = "/tf/notebooks/CEIA/computer-vision-2/tp-final/runs"
TRAIN_PROJECT_PATH = os.path.join(RUNS_PATH, "train")
TRAIN_SCRIPT = os.path.join(BASE_PATH, "yolov7/train.py")
DATASET_PATH = "/tf/datasets/final-dataset/yolov7-dataset"
DATA_YAML = os.path.join(DATASET_PATH, "data.yaml")

NUM_BATCHES = 4
NUM_EPOCHS = 2
FREEZE_LAYERS = 50 # backbone
IMG_SIZE = 400
experiment = "test"

torch.cuda.empty_cache() 

!python {TRAIN_SCRIPT} \
--project {TRAIN_PROJECT_PATH} \
--name {experiment} \
--batch {NUM_BATCHES} \
--epochs {NUM_EPOCHS} \
--data {DATA_YAML} \
--img-size {IMG_SIZE} {IMG_SIZE} \
--weights {WEIGHTS_FILE} \
--device 0 \
--freeze {FREEZE_LAYERS}

YOLOR 🚀 v0.1-112-g55b90e1 torch 1.12.1+cu102 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11177.25MB)

Namespace(adam=False, artifact_alias='latest', batch_size=4, bbox_interval=-1, bucket='', cache_images=False, cfg='', data='/tf/datasets/final-dataset/yolov7-dataset/data.yaml', device='0', entity=None, epochs=2, evolve=False, exist_ok=False, freeze=[50], global_rank=-1, hyp='./notebooks/CEIA/computer-vision-2/tp-final/models/yolov7/yolov7/data/hyp.scratch.p5.yaml', image_weights=False, img_size=[400, 400], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='test', noautoanchor=False, nosave=False, notest=False, project='/tf/notebooks/CEIA/computer-vision-2/tp-final/runs/train', quad=False, rect=False, resume=False, save_dir='/tf/notebooks/CEIA/computer-vision-2/tp-final/runs/train/test', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=4, upload_dataset=False, v5_metric=False, weights='/tf/notebooks/CEIA/computer-vision-2/tp-final/models/yolov7/yolov

In [6]:
# run tensorboard
# !tensorboard --logdir=/tf/notebooks/CEIA/computer-vision-2/tp-final/runs/train --host 0.0.0.0

In [ ]:
TEST_PROJECT_PATH = os.path.join(BASE_PATH, "runs/test")
TEST_SCRIPT = os.path.join(BASE_PATH, "yolov7/detect.py")
TEST_WEIGHTS = os.path.join(TRAIN_PROJECT_PATH, f"{experiment}/weights/best.pt")
TEST_DATA_PATH = os.path.join(DATASET_PATH, "test/images")

CONFIDENCE_THRESHOLD = 0.1

!python {TEST_SCRIPT} \
 --project {TEST_PROJECT_PATH} \
 --name {experiment} \
 --weights {TEST_WEIGHTS} \
 --conf-thres {CONFIDENCE_THRESHOLD} \
 --source {TEST_DATA_PATH}

In [ ]:
LOGS_PATH = os.path.join(BASE_PATH, "yolov7-pretrained")
RESULTS_PATH = os.path.join(BASE_PATH, "yolov7-pretrained")

try:
    original_umask = os.umask(0)
    os.mkdir(PRETRAINED_PATH, mode=0o775)
except FileExistsError:
    pass
finally:
    os.umask(original_umask)

In [7]:
RUNS_PATH = "/tf/runs"
TRAIN_RUNS = os.path.join(RUNS_PATH, "train")
TEST_RUNS = os.path.join(RUNS_PATH, "test")

results = {}
experiment_results_file = os.path.join(TRAIN_RUNS, f"retry-exp-1/results.txt")
# experiment_results_file = os.path.join(TRAIN_RUNS, f"{experiment}/results.txt")
with open(experiment_results_file) as f:
    line = " ".join(f.readline().split())  # read line and remove extra spaces
    experiment_results = line.split(" ")
    results[experiment] = {
        "train": {
            "precision": experiment_results[8],
            "recall": experiment_results[9],
            "mAP@0.5": experiment_results[10],
        }
    }


# assume you have the following dictionary
sampleDict = { "name": "Emma", "rollNumber": 5 }
with open("student.json", "w") as write_file:
    json.dump(results, write_file, indent=4)

FileNotFoundError: [Errno 2] No such file or directory: '/tf/runs/train/retry-exp-1/results.txt'

In [5]:


experiments = [f.path.split("/")[-1] for f in os.scandir(TRAIN_RUNS)]
last_experiment = natsorted(experiments)[-1]

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.1, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp-13', no_trace=False, nosave=False, project='/tf/notebooks/CEIA/computer-vision-2/tp-final/runs/test', save_conf=False, save_txt=False, source='/tf/datasets/final-dataset/yolov7-dataset/test/images', update=False, view_img=False, weights=['/tf/notebooks/CEIA/computer-vision-2/tp-final/runs/train/exp-13/weights/best.pt'])
YOLOR 🚀 v0.1-112-g55b90e1 torch 1.12.1+cu102 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11177.25MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
/usr/local/lib/python3.8/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 la

In [ ]:
#display inference on ALL test images

import glob
from IPython.display import Image, display

i = 0
limit = 1 # max images to print
for imageName in glob.glob('/content/yolov7/runs/detect/exp32/*.jpg'): #assuming JPG
    if i < limit:
      display(Image(filename=imageName))
      print("\n")
    i = i + 1
    

# Reparameterize for Inference

https://github.com/WongKinYiu/yolov7/blob/main/tools/reparameterization.ipynb

# OPTIONAL: Deployment

To deploy, you'll need to export your weights and save them to use later.

In [ ]:
# optional, zip to download weights and results locally

!zip -r export.zip runs/detect
!zip -r export.zip runs/train/exp/weights/best.pt
!zip export.zip runs/train/exp/*

# OPTIONAL: Active Learning Example

Once our first training run is complete, we should use our model to help identify which images are most problematic in order to investigate, annotate, and improve our dataset (and, therefore, model).

To do that, we can execute code that automatically uploads images back to our hosted dataset if the image is a specific class or below a given confidence threshold.


In [ ]:
# # setup access to your workspace
# rf = Roboflow(api_key="YOUR_API_KEY")                               # used above to load data
# inference_project =  rf.workspace().project("YOUR_PROJECT_NAME")    # used above to load data
# model = inference_project.version(1).model

# upload_project = rf.workspace().project("YOUR_PROJECT_NAME")

# print("inference reference point: ", inference_project)
# print("upload destination: ", upload_project)

In [ ]:
# # example upload: if prediction is below a given confidence threshold, upload it 

# confidence_interval = [10,70]                                   # [lower_bound_percent, upper_bound_percent]

# for prediction in predictions:                                  # predictions list to loop through
#   if(prediction['confidence'] * 100 >= confidence_interval[0] and 
#           prediction['confidence'] * 100 <= confidence_interval[1]):
        
#           # upload on success!
#           print(' >> image uploaded!')
#           upload_project.upload(image, num_retry_uploads=3)     # upload image in question

# Next steps

Congratulations, you've trained a custom YOLOv7 model! Next, start thinking about deploying and [building an MLOps pipeline](https://docs.roboflow.com) so your model gets better the more data it sees in the wild.